# PPD: Programação com CUDA

Hélio - DC/UFSCar - 2023

https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#memory-hierarchy

# Gerenciamento de memória na GPU

Um dos aspectos fundamentais na programação usando GPUs é a **comunicação CPU / GPU**. Mais especificamente, a cópia de dados da memória RAM para a memória do dispositivo, para que sejam acessados pelas *threads* dos *kernels*, e vice-versa, trazendo para a RAM os dados resultados das manipulações em GPU.

Como veremos, essa transferência pode ser feita de maneira explícita, controlada pelo programa executando em CPU, ou pode ser feita de forma forma transparente, apoiada pelos mecanismos de endereçamento de E/S mapeada em memória. De todo modo, cabe ao programador especificar quais são as áreas de memória utilizadas na GPU.

Mas por que é preciso preocupar-se com áreas de memória em GPU?

A obtenção de dados de entrada a partir de arquivos, ou da rede, por exemplo, é feita pelo programa executando em CPU, colocando-os na memória RAM. Para processamento em GPU, contudo, esses dados precisam ser transferidos para a memória deste dispositivo.

Antes de fazer isso, é preciso **reservar espaço** na memória da GPU.

Tendo alocado os espaços, é possível **copiar** dados da memória RAM, comumente referida como memória do *host*, para a memória do dispositivo (*device*).

Uma vez que novos dados tenham sido gerados pelo processamento em GPU, é preciso **copiar** os dados de interesse da memória deste dispositivo para a memória RAM, de forma que possam ser salvos em arquivos, transmitidos ou tratados de alguma outra forma.

Na programação com CUDA, esse gerenciamento de memória pode ser feito de forma simplificada, com funções semelhantes à alocação e cópia em memória em C.


1.   **Alocar espaço de memória na GPU**

      cudaError_t [cudaMalloc](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__MEMORY.html#group__CUDART__MEMORY_1g37d37965bfb4803b6d4e59ff26856356) ( void\*\* devPtr , size_t size)

2.   **Transferir os dados entre CPU (RAM) e (memória da) GPU**

     cudaError_t [cudaMemcpy](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__MEMORY.html#group__CUDART__MEMORY_1gc263dbe6574220cc776b45438fc351e8) ( void\* **dst**, const void\* **src**, size_t count, cudaMemcpyKind kind)

3.   **Retornar os dados para a memória da CPU (RAM)**

     cudaError_t **cudaMemcpy**( void\* dst, const void\* src, size_t count, cudaMemcpyKind kind)

4.   **Liberar espaço de memória alocado na GPU**

     cudaError_t **cudaFree**( void\* ptr)




O exemplo a seguir, extraído de https://developer.nvidia.com/blog/easy-introduction-cuda-c-and-c, ilustra uma implementação do programa SAXPY
(*Single-precision A * X Plus Y*), onde se pode ver a manipulação de memória com alocação, cópias de e para a GPU e liberação do espaço.



In [ ]:
%%writefile saxpy.cu

#include <stdio.h>

__global__
void saxpy(int n, float a, float *x, float *y)
{
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  if (i < n)
    y[i] = a*x[i] + y[i];
}

int main(void)
{
  int N = 1<<20;
  float *x, *y, *d_x, *d_y;

  x = (float*)malloc(N*sizeof(float));
  y = (float*)malloc(N*sizeof(float));

  // alocação de memória na GPU
  cudaMalloc(&d_x, N*sizeof(float));
  cudaMalloc(&d_y, N*sizeof(float));

  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Cópia dos dados da memória RAM para a memória do dispositivo
  cudaMemcpy(d_x, x, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_y, y, N*sizeof(float), cudaMemcpyHostToDevice);

  // Perform SAXPY on 1M elements
  saxpy<<<(N+255)/256, 256>>>(N, 2.0f, d_x, d_y);

  // Cópia dos dados da memória da GPU para a memória RAM
  cudaMemcpy(y, d_y, N*sizeof(float), cudaMemcpyDeviceToHost);

  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = max(maxError, abs(y[i]-4.0f));
  printf("Max error: %f\n", maxError);

  // Liberação das áreas de memória alocadas da GPU
  cudaFree(d_x);
  cudaFree(d_y);
  free(x);
  free(y);
}

Overwriting saxpy.cu


In [ ]:
! nvcc saxpy.cu -o saxpy -O3
! ./saxpy

Max error: 0.000000


# Alocação de memória na GPU

Há vários aspectos a notar no programa acima. Primeiro, com relação à **alocação** e à **liberação** de área de memória no dispositivo, vê-se as funções [cudaMalloc](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__MEMORY.html#group__CUDART__MEMORY_1g37d37965bfb4803b6d4e59ff26856356) (22 e 23) e, ao final do programa (46 e 47) chamadas a **cudaFree**, cuja utilização é bem semelhante ao que ocorre com *malloc* e *free* na alocação dinâmica em C.

<br>

# Nomeação de variáveis

A **nomeação** das variáveis também merece um comentário. Por questões de organização do código, é comum que as variáveis que vão referir-se a espaços de **endereçamento dentro do dispositivo** (GPU) sejam nomeadas com o prefixo **d_**. Isso não é obrigatório, mas é bastante comum na programação com aceleradores. Nessa mesma lógica, alguns programadores costumam nomear as variáveis correspondentes do programa em CPU com o prefixo **h_**.

<br>

# Cópia de dados CPU / GPU

Passando à atribuição de valores às variáveis e estruturas que serão manipuladas em GPU, nas linhas 31, 32 e 38, vê-se exemplos de uso da função [cudaMemcpy](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__MEMORY.html#group__CUDART__MEMORY_1gc263dbe6574220cc776b45438fc351e8).

```c
__host__ ​cudaError_t cudaMemcpy ( void* dst, const void* src, size_t count, cudaMemcpyKind kind )
```
* **dst**: endereço de memória destino
* **src**: endereço de memória de origem
* **count**: número de bytes a copiar
* **kind**: tipo (sentido) da transferência (comumente ***cudaMemcpyHostToDevice*** ou ***cudaMemcpyDeviceToHost***)

Como se pode observar, o primeiro parâmetro é o **destino** da cópia, seguido da **origem** e o número de bytes a copiar. Já o parâmetro *kind* indica o **sentido** da transferência, comumente indicando cópia do *host* (RAM) para o dispositivo (*device*), ou vice-versa.

<br>

Neste exemplo específico, há inicialmente a transferência (cópia) do conteúdo dos 2 vetores da memória RAM para a área pré-alocada na GPU (31 e 32) e, ao final do processamento na GPU, a cópia apenas do vetor modificado na GPU para a memória RAM (38). É claro que não é preciso copiar de volta o vetor x neste caso, já que ele não foi alterado na GPU.

É importante ressaltar que essas operações de transferência entre áreas de memória do *host* e do *device* são [síncronas](https://docs.nvidia.com/cuda/cuda-runtime-api/api-sync-behavior.html#api-sync-behavior__memcpy-sync). Ou seja, o código só passa para a linha seguinte após a transferência ser realizada.

Há uma série de outras funções **cuda** para [gerenciamento de memória](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__MEMORY.html), incluindo suporte para transfências bi-dimensionais e assíncronas ([cudaMemcpyAsync](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__MEMORY.html#group__CUDART__MEMORY_1g85073372f776b4c4d5f89f7124b7bf79)).

<br>

# Passagem de parâmetros para a função do *kernel*

Outro aspecto fundamental a observar neste exemplo é a **passagem de parâmetros** para a função do *kernel*.

```c
   // Declaração da função
   __global__ void saxpy(int n, float a, float *x, float *y)
  ...
  // invocação da função como um kernel
  saxpy<<<(N+255)/256, 256>>>(N, 2.0f, d_x, d_y);
```
Como se vê, a função *saxpy* tem 4 parâmetros, sendo um valor inteiro, um valor em ponto flutuante e dois ponteiros para valores em ponto flutuante, os vetores neste caso.

A passagem de **valores** (fixos ou contidos em variáveis) na invocação de um *kernel* é simples e é resolvida pelos mecanismos de CUDA. Já a passagem de ponteiros requer atenção. É claro que não adiantaria passar à função do *kernel* endereços no espaço de endereçamento da memória RAM (\*).

    (*) salvo se tratar-se de espaço de memória do dispositivo mapeado em RAM, o que estudaremos posteriormente.

Assim, neste exemplo, os endereços que são passados como parâmetro são os endereços que alocamos anteriormente na memória do dispositivo, com cudaMalloc!

# Ativação do *kernel*

Ainda na ativação do *kernel*, antes dos parâmetros é definida a organização das *threads* que serão usadas na execução do código.

Neste caso, optou-se por blocos com 256 *threads*. Assim, dado que são 1M elementos (1\<\<20), o número de blocos pode ser calculado dividindo-se (N+255) por 256.

```c
  saxpy<<<(N+255)/256, 256>>>( ... );
```


# CUDA Unified Memory

Como uma evolução do modelo de memória e da comunicação *host* / *device*, a versão 6 de CUDA implantou o suporte para o que é chamado de [*Unified Memory*](https://developer.nvidia.com/blog/unified-memory-in-cuda-6/). \([*](https://developer.nvidia.com/blog/unified-memory-cuda-beginners/)\)

Fisicamente, GPUs são conectadas ao sistema hospedeiro através de algum barramento, tipicamente o PCI-Express. As áreas de memória acessíveis diretamente por CPUs e pelos processadores da GPU são distintas, sendo a RAM para as CPUs e a memória presente na GPU para os processadores deste dispositivo.

Assim, uma estratégia comum de programação em GPUs consiste em alocar espaços de memória em GPU, copiar dados necessários na memória deste dispositivo e copiar de volta à RAM dados relevantes que foram produzidos pelo dispositivo.

Já o mecanismo de memória unificada (*unified memory*) de CUDA cria um conjunto de áreas de memória gerenciadas que são compartilhadas entre CPU e GPU. Uma área de memória alocada com esse mecanismo é **acessível diretamente**, tanto pelo código em CPU quanto pelo código em GPU, usando **o mesmo ponteiro**.

De maneira simplificada, para alocar áreas de memória (variáveis) compartilhadas entre CPU e GPU, basta usar a função [cudaMallocManaged](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__MEMORY.html#group__CUDART__MEMORY_1gd228014f19cc0975ebe3e0dd2af6dd1b)().

```c
char *data;
cudaMallocManaged(&data, N);
```

Os dados alocados em áreas de memória unificada são automaticamente migrados entre a memória RAM e a memória do dispositivo, de forma que cada código (de CPU ou de GPU) veja esses dados como se fossem locais.

<br>

[1] https://developer.nvidia.com/blog/unified-memory-cuda-beginners/

    When code running on a CPU or GPU accesses data allocated this way (often called CUDA managed data),
    the CUDA system software and/or the hardware takes care of migrating memory pages to the memory of the
    accessing processor.

<br>

Vejamos uma ilustração dos 2 modelos de acesso à memória:

 <img src="https://developer-blogs.nvidia.com/wp-content/uploads/2013/11/unified_memory.png">

Como resultado, simplifica-se o uso de memória e das interações CPU/GPU em aplicações CUDA.

A figura a seguir apresenta 2 versões de um código que realiza a leitura de dados de arquivo, chama uma função de ordenação dos dados e os manipula, antes de gravar os dados em arquivo novamente.

<img src="https://developer-blogs.nvidia.com/wp-content/uploads/2013/11/simplified_memory_mananagement_code-e1384437984510.png">

A alocação dos dados é feita via [cudaMallocManaged](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__MEMORY.html#group__CUDART__MEMORY_1gd228014f19cc0975ebe3e0dd2af6dd1b)(), de forma que os dados alocados são acessíveis manipulando os mesmos ponteiros tanto no programa em CPU quanto na GPU.

Uma **etapa a mais** necessária neste caso é a **sincronização** após a ativação do *kernel*, para garantir que as operações que manipulam os dados já foram concluídas, evitando transferências de dados que ainda estão em uso na GPU.

Isso é feito com a chamada [cudaDeviceSynchronize](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__DEVICE.html#group__CUDART__DEVICE_1g10e20b05a95f638a4071a655503df25d)(), que bloqueia até que o dispositivo tenha concluído todas as operações requisitadas.

Toda a complexidade da migração dos dados é tratada por CUDA.


Vejamos um exemplo de código, apresentado em \[[1](https://developer.nvidia.com/blog/unified-memory-cuda-beginners/)\] que realiza a soma de 2 vetores, declarados como memória unificada.

\[1\] https://developer.nvidia.com/blog/unified-memory-cuda-beginners/

In [ ]:
    %%writefile uni-sum.cu

    #include <iostream>
    #include <math.h>

    // CUDA kernel to add elements of two arrays
    __global__
    void add(int n, float *x, float *y)
    {
      int index = blockIdx.x * blockDim.x + thr:eadIdx.x;
      int stride = blockDim.x * gridDim.x;
      for (int i = index; i < n; i += stride)
        y[i] = x[i] + y[i];
    }

    int main(void)
    {
      int N = 1<<20;
      float *x, *y;

      // Allocate Unified Memory -- accessible from CPU or GPU
      cudaMallocManaged(&x, N*sizeof(float));
      cudaMallocManaged(&y, N*sizeof(float));

      // initialize x and y arrays on the host
      for (int i = 0; i < N; i++) {
        x[i] = 1.0f;
        y[i] = 2.0f;
      }

      // Launch kernel on 1M elements on the GPU
      int blockSize = 256;
      int numBlocks = (N + blockSize - 1) / blockSize;
      add<<<numBlocks, blockSize>>>(N, x, y);

      // Wait for GPU to finish before accessing on host
      cudaDeviceSynchronize();

      // Check for errors (all values should be 3.0f)
      float maxError = 0.0f;
      for (int i = 0; i < N; i++)
        maxError = fmax(maxError, fabs(y[i]-3.0f));
      std::cout << "Max error: " << maxError << std::endl;

      // Free memory
      cudaFree(x);
      cudaFree(y);

      return 0;
    }

Writing uni-sum.cu


In [ ]:
! if [ ! uni-sum -nt uni-sum.cu ]; then nvcc uni-sum.cu -o uni-sum -O3; fi
! ./uni-sum

Max error: 0


## Aspectos do uso de *unified memory*

Há 2 aspectos principais no uso de áreas de memória unificadas. Primeiro, o modelo de programação torna-se mais simples, sem ter que incorporar as cópias de dados entre CPU (RAM) e GPU.

Outro aspecto é que a cópia dos dados entre CPU e GPU sob demanda provê a localidade dos acessos e os ganhos de desempenho que isso proporciona, de forma transparente para a aplicação. O uso de memórias de alta velocidade, contudo, é um requisito para que o mecanismo de transferências sob demanda consiga prover todo o desempenho de GPUs modernas.

<br>

### Memory-mapped I/O

Cabe lembrar, contudo, dos mecanismos de acesso direto à memória (DMA) e de mapeamentos de espaços de endereçamentos de memória para entrada e saída ([*memory mapped I/O*](https://en.wikipedia.org/wiki/Memory-mapped_I/O)).

*Memory-mapped I/O* usa o **mesmo espaço de endereçamento** que é usado para acessar a memória RAM para acessar também o conteúdo de registradores de controle e de áreas de memória dentro de dispositivos de E/S. Assim, as mesmas instruções que fazem acesso à memória podem ser usadas para o acesso a dispositivos, sem modificações no código.

Isso é possível porque mecanismos providos pelo sistema de interligação de CPUs, memória e dispositivos de E/S no computador permitem a **reserva de faixas de endereços** que levariam à memória para uso em transferências para controladores específicos interligados aos barramentos PCI-Express, por exemplo.

Feito isso, cabe a cada dispositivo de E/S  monitorar os endereços nos acessos ao barramento que leva à memória e tratar as transferências aos endereços que reservou.

Há ainda questões relacionadas ao mapeamento de páginas virtuais associadas às áreas de memória reservadas para as variáveis alocadas com o modo unificado.

<br>

[1] https://developer.nvidia.com/blog/unified-memory-cuda-beginners/

    On Pascal and later GPUs, managed memory may not be physically allocated when cudaMallocManaged() returns;
    it may only be populated on access (or prefetching). In other words, pages and page table entries
    may not be created until they are accessed by the GPU or the CPU. The pages can migrate to any processor’s
    memory at any time, and the driver employs heuristics to maintain data locality and prevent excessive page faults.

<br>

### Aspectos de desempenho nos modelos de acesso à memória pela GPU

Embora possa prover transferências em altas taxas e de maneira transparente para a aplicação, o **modelo de memória unificada talvez perca em desempenho para programas que usam o modelo de alocação original mas que conhecem seus padrões de acesso à memória e que usem estratégias como a sobreposição de operações de transferência de dados com processamento**.

Outras considerações sobre desempenho e aspectos do funcionamento do mecanismo de mapeamento de memória podem ser vistas em https://developer.nvidia.com/blog/unified-memory-cuda-beginners/ e em https://developer.nvidia.com/blog/maximizing-unified-memory-performance-cuda/.

Entre outros aspectos ressaltados por essas referências está a constatação que para maximizar o desempenho, os dados devem ser mantidos o mais próximo possível da GPU.

Em situações em que o volume de dados a manipular é maior do que o espaço de endereçamento da GPU, o uso de memória unificada simplifica os acessos, deixando ao sistema o gerenciamento de quais partes dos dados vão estar efetivamente copiados na memória da GPU. Isso nem sempre pode prover melhor desempenho do que deixar ao programador copiar explicitamente na GPU os dados que serão manipulados.


# Suporte a *managed memory*

Para saber se uma GPU tem suporte à alocação de memória no modelo unificado, com cudaMallocManaged, é possível analisar as propriedades da GPU. Essas informações podem ser obtidas com a chamada [cudaGetDeviceProperties](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__DEVICE.html#group__CUDART__DEVICE_1g1bf9d625a931d657e08db2b4391170f0) e observando o campo managedMemory.

```c
struct cudaDeviceProp {
  char name[256];
  ...
  int cudaDeviceProp::managedMemory; // Device supports allocating managed memory on this system
  ...
}
```


In [ ]:
%%writefile man-mem.cu

#include <stdio.h>

int main(void)
{
	cudaSetDevice(0);
	cudaDeviceProp prop;
	cudaGetDeviceProperties(&prop,0);

 	printf("Modelo do Device: %s\n",prop.name);
  printf("Número de SMs: %d\n",prop.multiProcessorCount);

  printf("managedMemory: %d\n",prop.managedMemory);

	return 0;
}

Writing man-mem.cu


In [ ]:
! if [ ! man-mem -nt man-mem.cu ]; then nvcc man-mem.cu -o man-mem  -Wno-deprecated-gpu-targets -gencode=arch=compute_37,code=sm_37 ; fi
! ./man-mem

Modelo do Device: Tesla T4
Número de SMs: 40
managedMemory: 1


# Exemplo de programa CUDA usando cudaMallocManaged


O exemplo a seguir, extraído de https://developer.nvidia.com/blog/even-easier-introduction-cuda, ilustra o funcionamento dos mecanismos de alocação e cópia em memória envolvendo a GPU, com *Unified Memory*.

Neste exemplo, é feita a soma sequencial, em CPU, de 1M elementos de 2 vetores arnazenados em memória RAM.

A primeira versão do programa é um código sequencial em CPU.

In [ ]:
%%writefile vec-sum.c

#include <stdio.h>
#include <stdlib.h>
#include <math.h>

// function to add the elements of two arrays
void add(int n, float *x, float *y)
{
  for (int i = 0; i < n; i++)
      y[i] = x[i] + y[i];
}

int main(void)
{
  int N = 1<<20; // 1M elements

  float *x = malloc (N * sizeofapresenta(float));
  float *y = malloc (N * sizeof(float));

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the CPU
  add(N, x, y);

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));

  printf("Max error: %f\n", maxError);

  // Free memory
  free(x);
  free(y);

  return 0;
}

Overwriting vec-sum.c


In [ ]:
# ! gcc -Wall vec-sum.c -o vec-sum -lm && ./vec-sum
! if [ ! vec-sum -nt vec-sum.c ]; then gcc -Wall vec-sum.c -o vec-sum -lm ; fi
! ./vec-sum

Já dá para supor que, com muitos processadores operando em paralelo na execução do mesmo código, deve ser vantajoso gerar uma versão deste código para GPU.

Um primeiro passo para isso é gerar uma versão da função de soma na forma de um *kernel*, incluindo o prefixo **\_\_global\_\_**.

```c
// CUDA Kernel para a adição dos elementos de 2 vetores em GPU
__global__
void add(int n, float *x, float *y)
{
  for (int i = 0; i < n; i++)
      y[i] = x[i] + y[i];
}
```

Outro aspecto necessário é a alocação de memória na GPU para conter os vetores que serão somados.

O modelo de memória unificado (*Unified Memory*) em CUDA provê um espaço de endereçamento único que é acessível por todas as GPUs e CPUs do sistema. A alocação de memória nesse espaço é feita com a função [cudaMallocManaged](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__MEMORY.html#group__CUDART__MEMORY_1gd228014f19cc0975ebe3e0dd2af6dd1b), que permite que o espaço alocado seja acessível pelo código no *host* e pelo código na GPU. A posterior liberação deste espaço é feita com a chamada cudaFree().

Desta forma, basta substituir as funções de alocação e liberação de memória no código:
```c
  // Aloca memória Unificada, acessível por GPUs e CPUs
  float *x, *y;
  cudaMallocManaged(&x, N*sizeof(float));
  cudaMallocManaged(&y, N*sizeof(float));

  ...

  // Free memory
  cudaFree(x);
  cudaFree(y);
```

Para a execução do código em GPU, a chamada da função dá origem à invocação do *kernel*.

Um aspecto relevante na ativação do *kernel* é a seleção do número de *threads* que serão usadas na execução.

Do modo em que está implementada a função de soma como um *kernel*, todas as iterações serão feitas pela *thread* que a executar.
Assim, no exemplo a seguir, o *kernel* é ativado com apenas 1 *thread*. O número de blocos é especificado em 1, com apenas 1 thread por bloco.

```c
add<<<1, 1>>>(N, x, y);
```
É claro, contudo que, deste modo, não estamos explorando o potencial de paralelismo dos múltiplos processadores da GPU. Idealmente, devemos considerar blocos de processadores atuando em paralelo, dividindo as iterações do laço na função do kernel. Havendo *cores* suficientes na GPU, cada umd deles ficaria encarregado de manipular um único elemento daquele laço (*for*). Trataremos disso posteriormente.

Outro aspecto a observar é o assincronismo na execução dos *kernels* pela GPU. Uma vez ativado o *kernel*, a execução do código prossegue na CPU. Assim, antes de poder acessar na memória os dados que serão produzidos pelo código da GPU, é preciso executar uma operação de sincronização.

Isso pode ser feito com a chamada cudaDeviceSynchronize().


In [ ]:
# %%cu
%%writefile gpu-sum.cu

#include <stdio.h>
#include <math.h>

// Kernel function to add the elements of two arrays
__global__
void add(int n, float *x, float *y)
{
  for (int i = 0; i < n; i++)
    y[i] = x[i] + y[i];
}

int main(void)
{
  int N = 1<<20;
  float *x, *y;

  // Allocate Unified Memory – accessible from CPU or GPU
  cudaMallocManaged(&x, N*sizeof(float));
  cudaMallocManaged(&y, N*sizeof(float));

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the GPU
  add<<<1, 1>>>(N, x, y);

  // Wait for GPU to finish before accessing on host
  cudaDeviceSynchronize();

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));

  printf("Max error: %f\n", maxError);

  // Free memory
  cudaFree(x);
  cudaFree(y);

  return 0;
}

Writing gpu-sum.cu


In [ ]:
! if [ ! gpu-sum -nt gpu-sum.cu ]; then nvcc gpu-sum.cu -o gpu-sum  -Wno-deprecated-gpu-targets -gencode=arch=compute_37,code=sm_37 ; fi
! ./gpu-sum

Max error: 0.000000


# Sobre acesso aos dados com *unified memory*

https://www.cs.ucr.edu/~amazl001/teaching/cs147/S21/slides/11-Pinned_UnifiedMemory.pdf

UNIFIED MEMORY ON PRE-PASCAL

Code example explained
```c
cudaMallocManaged(&ptr, . . . ) ;     // Pages are populated in GPU memory
*ptr = 1;                             // CPU page fault: data migrates to CPU
qsort<<<...>>>(ptr);                  // Kernel launch: data migrates to GPU
```
* GPU always has address translation during the kernel execution
* Pages allocated before they are used – cannot oversubscribe GPU
* Pages migrate to GPU only on kernel launch – cannot migrate on-demand

**Kernel launch triggers bulk page migrations**

<br>

UNIFIED MEMORY ON PASCAL

Now supports GPU page faults
```c
cudaMallocManaged(&ptr, . . . ) ;   // Empty, no pages anywhere (similar to malloc)
*ptr = 1;                           // CPU page fault: data allocates on CPU
qsort<<<...>>>(ptr);                // GPU page fault: data migrates to GPU
```
* If GPU does not have a VA translation, it issues an interrupt to CPU
* Unified Memory driver could decide to map or migrate depending on heuristics
* Pages populated and data migrated on first touch

**True on-demand page migrations**

<br>

Pela descrição a seguir, vê-se que GPUs a partir da linha Pascal possuem um mecanismo que detecta falta de páginas na própria GPU e é capaz de parar a execução do kernel até que os dados (páginas de memória) necessários sejam carregados para o dispositivo.

Assim, não há uma cópia automática dos dados antes da invocação do *kernel*, mas essa transferência ocorre sob demanda ao ser identificada uma falta de página. Deste modo, o tempo de execução do *kernel* é maior, ao incluir o tempo de transferência.

<br>

\[1\] https://developer.nvidia.com/blog/unified-memory-cuda-beginners/

*Unlike the pre-Pascal GPUs, the Tesla P100 supports hardware page faulting and migration. So in this case the runtime doesn’t automatically copy all the pages back to the GPU before running the kernel. The kernel launches without any migration overhead, and when it accesses any absent pages, the GPU stalls execution of the accessing threads, and the Page Migration Engine migrates the pages to the device before resuming the threads.*

*This means that the cost of the migrations is included in the kernel run time when I run my program on the Tesla P100 (2.1192 ms). In this kernel, every page in the arrays is written by the CPU, and then accessed by the CUDA kernel on the GPU, causing the kernel to wait on a lot of page migrations. That’s why the kernel time measured by the profiler is longer on a Pascal GPU like Tesla P100.*

<br>

## O que pode ser feito para melhorar o desempenho ao usar *unified memory*

Ainda segundo a referência anterior \[1\], algumas estratégias podem ser adotadas para que o mecanismo de transferência dos dados via falta de páginas durante a execução do *kernel* não prejudique seu desempenho:

* Realizar a inicialização dos dados dos vetores na GPU, o que pode ser feito por uma outra função de *kernel*. Neste caso, a área de memória (páginas) já seria alocada diretamente na GPU, sem necessidade de transferência no início da execução do *kernel* manipula os dados;

```c
__global__ void init(int n, float *x, float *y) {
  int index = threadIdx.x + blockIdx.x * blockDim.x;
  int stride = blockDim.x * gridDim.x;
  for (int i = index; i < n; i += stride) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }
}
```

* Realizar múltiplas execuções da função do kernel e calcular os valores médios e mínimo;

* Realizar o carregamento prévio dos dados na memória da GPU (*prefetching*) usando uma chamada específica para isso ([cudaMemPrefetchAsync](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__MEMORY.html#group__CUDART__MEMORY_1ge8dc9199943d421bc8bc7f473df12e42)).

```c
  // Prefetch the data to the GPU
  int device = -1;
  cudaGetDevice(&device);

  cudaMemPrefetchAsync(x, N*sizeof(float), device, NULL);
  cudaMemPrefetchAsync(y, N*sizeof(float), device, NULL);

  // Run kernel on 1M elements on the GPU
  int blockSize = 256;
  int numBlocks = (N + blockSize - 1) / blockSize;

  saxpy<<<numBlocks, blockSize>>>(N, 1.0f, x, y);
```

<br>

## Cuidado com concorrência nos acessos aos dados pelo código em CPU e em *kernels* na GPU

Apesar do mecanismo eficiente de detecção de faltas de páginas nas GPUs pós-Pascal, com capacidade de transferir páginas de memória com dados sob demanda, é claro que o acesso concorrente às áreas em memória unificada, pelo programa em CPU e por *kernel* em execução na GPU pode gerar problemas.

Ao tentar acessar na CPU dados em uso pelo *kernel* em execução, o programa causará uma falha de segmentação (*segmentation fault*).

<br>

## Sobre os benefícios de usar a memória unificada em GPUs Pascal e posteriores

Ainda segundo \[1], dada a eficiência do mecanismo de paginação na GPU, com transmissões mediante faltas de páginas, é possível que a estratégia de uso de memória unificada sirva para criar programas que requerem áreas de dados maiores do que o espaço disponível na GPU.

Além disso, para padrões de acesso a posições esparsas dos dados, o carregamento das páginas sob demanda pode ser mais eficiente do que carregar todo o conjunto de dados para a memória da GPU.

<br>

*The Benefits of Unified Memory on Pascal and Later GPUs*

*Starting with the Pascal GPU architecture, Unified Memory functionality is significantly improved with 49-bit virtual addressing and on-demand page migration. 49-bit virtual addresses are sufficient to enable GPUs to access the entire system memory plus the memory of all GPUs in the system. The Page Migration engine allows GPU threads to fault on non-resident memory accesses so the system can migrate pages on demand from anywhere in the system to the GPU’s memory for efficient processing.*

*In other words, Unified Memory transparently enables oversubscribing GPU memory, enabling out-of-core computations for any code that is using Unified Memory for allocations (e.g. cudaMallocManaged()). It “just works” without any modifications to the application, whether running on one GPU or multiple GPUs.*
...
*Demand paging can be particularly beneficial to applications that access data with a sparse pattern. In some applications, it’s not known ahead of time which specific memory addresses a particular processor will access. Without hardware page faulting, applications can only pre-load whole arrays, or suffer the cost of high-latency off-device accesses (also known as “Zero Copy”). But page faulting means that only the pages the kernel accesses need to be migrated.*


# ... work in progress... sobre pilha, strack frame e afins...

## Sobre pilha, recursividade e afins...

https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#call-stack


On devices of compute capability 2.x and higher, the size of the call stack can be queried usingcudaDeviceGetLimit() and set using cudaDeviceSetLimit().

When the call stack overflows, the kernel call fails with a stack overflow error if the application is run via a CUDA debugger (CUDA-GDB, Nsight) or an unspecified launch error, otherwise. When the compiler cannot determine the stack size, it issues a warning saying Stack size cannot be statically determined. This is usually the case with recursive functions. Once this warning is issued, user will need to set stack size manually if default stack size is not sufficient.

<br>

https://docs.nvidia.com/cuda/cuda-c-programming-guide/index,html#configuration-options

<br>

https://forums.developer.nvidia.com/t/what-is-the-maximum-cuda-stack-frame-size-per-kerenl/31449

<br>

https://stackoverflow.com/questions/74597338/how-is-stack-frame-managed-within-a-thread-in-cuda

<br>

How is stack frame managed within a thread in Cuda?

Suppose we have a kernel that invokes some functions, for instance:
```c
__device__ int fib(int n) {
    if (n == 0 || n == 1) {
        return n;
    } else {
        int x = fib(n-1);
        int y = fib(n-2);
        return x + y;
    }
    return -1;
}

__global__ void fib_kernel(int* n, int *ret) {
    *ret = fib(*n);
}
```

The kernel fib_kernel will invoke the function fib(), which internally will invoke two fib() functions. Suppose the GPU has 80 SMs, we launch exactly 80 threads to do the computation, and pass in n as 10. I am aware that there will be a ton of duplicated computations which violates the idea of data parallelism, but I would like to better understand the stack management of the thread.

According to the Documentation of Cuda PTX, it states the following:

    the GPU maintains execution state per thread, including a program counter and call stack

    The stack locates in local memory. As the threads executing the kernel, do they behave just like the calling convention in CPU? In other words, is it true that for each thread, the corresponding stack will grow and shrink dynamically?

    The stack of each thread is private, which is not accessible by other threads. Is there a way that I can manually instrument the compiler/driver, so that the stack is allocated in global memory, no longer in local memory?

    Is there a way that allows threads to obtain the current program counter, frame pointer values? I think they are stored in some specific registers, but PTX documentation does not provide a way to access those. May I know what I have to modify (e.g. the driver or the compiler) to be able to obtain those registers?

    If we increase the input to fib(n) to be 10000, it is likely to cause stack overflow, is there a way to deal with it? The answer to question 2 might be able to address this. Any other thoughts would be appreciated.

    cudagpudriver

asked Nov 28, 2022 at 7:33
Ethan L.

    Local memory is physically located in global memory –
    Abator Abetor
    Nov 28, 2022 at 8:40
    You could just compute fib iteratively, without worrying about stack size –
    Abator Abetor
    Nov 28, 2022 at 8:41

    Yeah, but local memory is not accessible by other threads. Is there a way to copy the stack frame in local memory and place it in global memory? Or we just modify the driver/compiler (not sure what has to be modified) so that the stack frame for each thread is allocated in global memory and is accessible by all other threads. This might lead to security issues but it is not that danger in terms of doing computation. –
    Ethan L.
    Nov 28, 2022 at 8:45
    With a debugger you can read the local memory of each thread, while single-stepping through the CUDA program. –
    Sebastian
    Nov 28, 2022 at 22:46

1 Answer

You'll get a somewhat better idea of how these things work if you study the generated SASS code from a few examples.

    As the threads executing the kernel, do they behave just like the calling convention in CPU? In other words, is it true that for each thread, the corresponding stack will grow and shrink dynamically?

The CUDA compiler will aggressively inline functions when it can. When it can't, it builds a stack-like structure in local memory. However the GPU instructions I'm aware of don't include explicit stack management (e.g. push and pop, for example) so the "stack" is "built by the compiler" with the use of registers that hold a (local) address and LD/ST instructions to move data to/from the "stack" space. In that sense, the actual stack does/can dynamically change in size, however the maximum allowable stack space is limited. Each thread has its own stack, using the definition of "stack" given here.

    Is there a way that I can manually instrument the compiler/driver, so that the stack is allocated in global memory, no longer in local memory?

Practically, no. The NVIDIA compiler that generates instructions has a front-end and a back-end that is closed source. If you want to modify an open-source compiler for the GPUs it might be possible, but at the moment there are no widely recognized tool chains that I am aware of that don't use the closed-source back end (ptxas or its driver equivalent). The GPU driver is also largley closed source. There aren't any exposed controls that would affect the location of the stack, either.

    May I know what I have to modify (e.g. the driver or the compiler) to be able to obtain those registers?

There is no published register for the instruction pointer/program counter. Therefore its impossible to state what modifications would be needed.

    If we increase the input to fib(n) to be 10000, it is likely to cause stack overflow, is there a way to deal with it?

As I mentioned, the maximum stack-space per thread is limited, so your observation is correct, eventually a stack could grow to exceed the available space (and this is a possible hazard for recursion in CUDA device code). The provided mechanism to address this is to increase the per-thread local memory size (since the stack exists in the logical local space).

answered Nov 28, 2022 at 19:18
Robert Crovella



    Thanks for your detailed reply. Suppose we have found out a way to convert local memory instructions into global memory instructions in SASS, e.g. LDL -> LDG, can we simply replace those local memory instructions with global memory instructions? Besides, for the memory address in SASS code, are they physical memory addresses? I think it is likely to be the actual physical address since it does not require address translation, but since local memory and global memory are all in DRAM, is the memory layout like "local mem": [0x0, 0x2FFFF..), "global mem": [0x2FFFF.., 0xFFFFF)? –
    Ethan L.
    Nov 29, 2022 at 2:56



## mais sobre a pilha...

https://stackoverflow.com/questions/7810740/where-does-cuda-allocate-the-stack-frame-for-kernels

Where does CUDA allocate the stack frame for kernels?
<br>
Asked 11 years, 5 months ago
Modified 10 years, 8 months ago
<br>

My kernel call fails with "out of memory". It makes significant usage of the stack frame and I was wondering if this is the reason for its failure.

When invoking nvcc with --ptxas-options=-v it print the following profile information:

    150352 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 59 registers, 40 bytes cmem[0]

Hardware: GTX480, sm20, 1.5GB device memory, 48KB shared memory/multiprocessor.

My question is where is the stack frame allocated: In shared, global memory, constant memory, ..?

I tried with 1 thread per block, as well as with 32 threads per block. Same "out of memory".

Another issue: One can only enlarge the number of threads resident to one multiprocessor if the total numbers of registers do not exceed the number of available registers at the multiprocessor (32k for my card). Does something similar apply to the stack frame size?

    cudastack

codetwiddler

asked Oct 18, 2011 at 16:41

Answers

Stack is allocated in local memory. Allocation is per physical thread (GTX480: 15 SM * 1536 threads/SM = 23040 threads). You are requesting 150,352 bytes/thread => ~3.4 GB of stack space. CUDA may reduce the maximum physical threads per launch if the size is that high. The CUDA language is not designed to have a large per thread stack.

In terms of registers GTX480 is limited to 63 registers per thread and 32K registers per SM.


<br><br>

https://forums.developer.nvidia.com/t/what-is-the-maximum-cuda-stack-frame-size-per-kerenl/31449



 njuffa November 18, 2013, 6:46pm 2

The compiler reports stack frame usage on a per-thread basis. The maximum stack frame size per thread for a given GPU is determined by (a) a hard architecture limit on the amount of local memory per thread (b) the amount of available GPU memory.

The architectural limit on the amount of local memory per thread is documented in the programming guide section G.1, table 12.
[url]http://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#features-and-technical-specifications[/url]

Available stack frame size per thread can then be approximated by

stack frame size available per thread =
min (amount of local memory per thread as documented in section G.1 table 12,
available GPU memory / number of SMs / maximum resident threads per SM)

The reason this is approximate is because there are various levels of allocation granularity that, best I know, are not documented and may vary from GPU to GPU. I do not know anything about your use case, but in general massive local memory usage would suggest to me that one might want to re-think the mapping of work to the GPU.

